In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import os
import warnings

# 파일 불러와서 열이름 바꾸기
korean, long_term_frgn, short_term_frgn, resident, card_cnt, card_amt, waste_cnt
(이거 예시는 card_amt임)

In [2]:
df = pd.read_csv("files\\data_preprocessing\\korean_preprocessing.csv", encoding='cp949')

# 시계열 피처단위로 변경
df['base_date'] = pd.to_datetime(df['base_date'])

# base_date를 index로 설정
df.index = df['base_date']
df.columns = ['base_date', 'emd_cd', 'korean']

In [3]:
# 필요 없는 열 제거
data = df.drop(['base_date'], 1)  
data

,emd_cd,korean
base_date,,
2018-01-31,50110250,7.209306e+06
2018-02-28,50110250,5.579125e+06
2018-03-31,50110250,7.615021e+06
2018-04-30,50110250,8.498349e+06
2018-05-31,50110250,8.916782e+06
...,...,...
2021-02-28,50130620,4.321265e+06
2021-03-31,50130620,4.744309e+06
2021-04-30,50130620,5.236706e+06


In [4]:
# 한 행정동의 행만 추출
sub_area = data['emd_cd'] == 50130320  ##각행정동코드

# base_date, card_amt 만 갖는 df 생성
sub_df = data[sub_area]
sub_df = sub_df.drop('emd_cd', 1)
sub_df

,korean
base_date,
2018-01-31,4.721659e+06
2018-02-28,3.901268e+06
2018-03-31,4.705604e+06
2018-04-30,5.776023e+06
2018-05-31,5.752806e+06
2018-06-30,5.239285e+06
2018-07-31,5.539686e+06
2018-08-31,5.805087e+06
2018-09-30,5.060449e+06


In [5]:
# series 형으로 변환
sub_df.to_dict('series')

{'korean': base_date
 2018-01-31    4.721659e+06
 2018-02-28    3.901268e+06
 2018-03-31    4.705604e+06
 2018-04-30    5.776023e+06
 2018-05-31    5.752806e+06
 2018-06-30    5.239285e+06
 2018-07-31    5.539686e+06
 2018-08-31    5.805087e+06
 2018-09-30    5.060449e+06
 2018-10-31    5.223508e+06
 2018-11-30    4.682964e+06
 2018-12-31    4.525793e+06
 2019-01-31    4.720533e+06
 2019-02-28    4.296346e+06
 2019-03-31    4.850608e+06
 2019-04-30    5.631029e+06
 2019-05-31    5.527185e+06
 2019-06-30    5.126170e+06
 2019-07-31    5.455830e+06
 2019-08-31    5.741279e+06
 2019-09-30    4.528188e+06
 2019-10-31    5.305492e+06
 2019-11-30    4.949676e+06
 2019-12-31    4.812545e+06
 2020-01-31    5.044235e+06
 2020-02-29    3.229860e+06
 2020-03-31    3.442256e+06
 2020-04-30    3.389788e+06
 2020-05-31    4.054357e+06
 2020-06-30    4.112311e+06
 2020-07-31    4.636048e+06
 2020-08-31    5.360256e+06
 2020-09-30    3.808723e+06
 2020-10-31    5.032260e+06
 2020-11-30    5.248213e+06

# ARIMA

In [6]:
# ARIMA에 적합한 float로 바꾸는 과정
sub_df_float = sub_df[:].astype(np.float)

In [7]:
# ADF 검정 - 정상성 확인하는 검정 (귀무 : 정상성 만족 x, 대립 : 정상성 만족)
from statsmodels.tsa.stattools import adfuller

result = adfuller(sub_df)
print('ADF Statistic : %f' % result[0])
print('p-value : %f' % result[1])
print('Critical Values :')
for key, value in result[4].items():
    print('\t%s: %3f' % (key, value))

ADF Statistic : -3.434186
p-value : 0.009847
Critical Values :
	1%: -3.600983
	5%: -2.935135
	10%: -2.605963


In [8]:
from pmdarima.arima import auto_arima

model_arima= auto_arima(sub_df,trace=True, error_action='ignore',suppress_warnings=True,stepwise=False,seasonal=True)

model_arima.fit(sub_df)

 ARIMA(0,0,0)(0,0,0)[1] intercept   : AIC=1252.267, Time=0.01 sec
 ARIMA(0,0,1)(0,0,0)[1] intercept   : AIC=1242.676, Time=0.10 sec
 ARIMA(0,0,2)(0,0,0)[1] intercept   : AIC=1243.870, Time=0.04 sec
 ARIMA(0,0,3)(0,0,0)[1] intercept   : AIC=1240.485, Time=0.07 sec
 ARIMA(0,0,4)(0,0,0)[1] intercept   : AIC=1242.374, Time=0.08 sec
 ARIMA(0,0,5)(0,0,0)[1] intercept   : AIC=1243.711, Time=0.10 sec
 ARIMA(1,0,0)(0,0,0)[1] intercept   : AIC=1240.716, Time=0.06 sec
 ARIMA(1,0,1)(0,0,0)[1] intercept   : AIC=1242.755, Time=0.05 sec
 ARIMA(1,0,2)(0,0,0)[1] intercept   : AIC=1244.488, Time=0.08 sec
 ARIMA(1,0,3)(0,0,0)[1] intercept   : AIC=1242.123, Time=0.12 sec
 ARIMA(1,0,4)(0,0,0)[1] intercept   : AIC=1243.592, Time=0.10 sec
 ARIMA(2,0,0)(0,0,0)[1] intercept   : AIC=1242.690, Time=0.05 sec
 ARIMA(2,0,1)(0,0,0)[1] intercept   : AIC=1244.735, Time=0.11 sec
 ARIMA(2,0,2)(0,0,0)[1] intercept   : AIC=1242.394, Time=0.16 sec
 ARIMA(2,0,3)(0,0,0)[1] intercept   : AIC=1243.812, Time=0.14 sec
 ARIMA(3,0

ARIMA(order=(0, 0, 3), scoring_args={}, seasonal_order=(0, 0, 0, 1),
      suppress_warnings=True)

In [9]:
# ARIMA 모델 
from statsmodels.tsa.arima_model import ARIMA
import statsmodels.api as sm

# (AR=2, 차분=1, MA=2) 파라미터로 ARIMA 모델을 학습합니다.
model = ARIMA(sub_df.korean.values, order=(0,0,3))   

#trend : constant를 가지고 있는지, c - constant / nc - no constant
#disp : 수렴 정보를 나타냄
model_fit = model.fit(trend='c', full_output=True, disp=True)
print(model_fit.summary())

                              ARMA Model Results                              
Dep. Variable:                      y   No. Observations:                   42
Model:                     ARMA(0, 3)   Log Likelihood                -614.700
Method:                       css-mle   S.D. of innovations         529202.907
Date:                Sat, 11 Sep 2021   AIC                           1239.399
Time:                        02:29:22   BIC                           1248.088
Sample:                             0   HQIC                          1242.584
                                                                              
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const       4.809e+06   1.82e+05     26.362      0.000    4.45e+06    5.17e+06
ma.L1.y        0.6972      0.123      5.654      0.000       0.456       0.939
ma.L2.y        0.1420      0.168      0.846      0.3

C:\Users\soeun\anaconda3\lib\site-packages\statsmodels\tsa\arima_model.py:472: FutureWarning: 
statsmodels.tsa.arima_model.ARMA and statsmodels.tsa.arima_model.ARIMA have
been deprecated in favor of statsmodels.tsa.arima.model.ARIMA (note the .
between arima and model) and
statsmodels.tsa.SARIMAX. These will be removed after the 0.12 release.

statsmodels.tsa.arima.model.ARIMA makes use of the statespace framework and
is both well tested and maintained.

To silence this warning and continue using ARMA and ARIMA until they are
removed, use:

import warnings
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARMA',
                        FutureWarning)
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARIMA',
                        FutureWarning)

  warnings.warn(ARIMA_DEPRECATION_WARN, FutureWarning)
C:\Users\soeun\anaconda3\lib\site-packages\statsmodels\base\model.py:547: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  warn

# 예측하기

In [10]:
# 2단위 이후의 예측결과
fore = model_fit.forecast(steps=2)
print(fore)

(array([5313649.95745581, 4808680.37727809]), array([529202.90694269, 645137.87577063]), array([[4276431.31933424, 6350868.59557739],
       [3544233.37570498, 6073127.37885119]]))
